In [1]:
# basic packages
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas.core.common import SettingWithCopyWarning
pd.options.mode.chained_assignment = None


import numpy as np
import os

In [2]:
# modeling packages
import statsmodels.api as sm
import statsmodels.formula as smf
from statsmodels.iolib.summary2 import summary_col # creating summary tables
import math

In [3]:
# main data
os.chdir('C:\\Users\\rbishwakarma\\Desktop\\value_proposition_in_python\\value_proposition')
df = pd.read_stata('final.dta')

In [4]:
# counting the missing values for fourrate and stem
df.stem.isnull().sum()
df.fourrate.isnull().sum()

1

In [5]:
# removing missing universities for the analysis
df = df.dropna(axis = 0, how = 'any', subset = ['retentionrate'])

# dropping values from the missing
df2 = df.dropna(axis = 0, how='any', subset=['stem'])
df2.reset_index(drop=True, inplace=True)

In [6]:
# Models
model1 = df[['instruction_FTE', 'academic_support_FTE', 'student_service_FTE', 'R1']]

model2 = df2[['instruction_FTE', 'academic_support_FTE', 'student_service_FTE',
          'research_FTE', 'SATtoACT_math_25', 'pell_grants', 'stem', 'R1']]

model3 = df2[['academic_support_FTE', 'student_service_FTE', 'research_FTE', 
          'SATtoACT_math_25', 'pell_grants', 'stem', 'salary_instruction', 'R1']]

model4 = df2[['academic_support_FTE', 'student_service_FTE', 'research_FTE', 'SATtoACT_math_25', 
          'pell_grants', 'stem', 'salary_instruction', 'female', 'white_percent', 'hispanic_percent', 
          'asian_percent', 'R1']]

model5 = df2[['instruction_FTE', 'academic_support_FTE', 'student_service_FTE', 'research_FTE',
          'SATtoACT_math_25', 'pell_grants', 'stem', 'female', 'white_percent', 'hispanic_percent',
          'asian_percent', 'R1']]

models = [model2, model3, model4, model5]

models_name = ['model2', 'model3', 'model4', 'model5']


#Compute weight for the WLS

In [7]:
# Model 1- weight
y1 = df['retentionrate']
ols = sm.OLS(y1, sm.add_constant(model1)).fit()
reisiduals = abs(ols.resid) # absolute value of the residuals

# step 2
ols2 = sm.OLS(reisiduals, sm.add_constant(df['students_FTE_total'])).fit()
fitted = ols2.predict()

# weight
weights = (1/fitted)**2
df.loc[:,'weight_4_model1'] = weights


In [8]:
# Models 3 - 9 weights
y = df2['retentionrate']
weight = df2['students_FTE_total']

# creating weights
for i in range(len(models)):
    # step 1
    ols = sm.OLS(y, sm.add_constant(models[i])).fit()
    reisdual = abs(ols.resid) 

    # step 2
    ols2 = sm.OLS(reisdual, sm.add_constant(weight)).fit()
    fitted = ols2.predict()

    # weight
    weights = (1/fitted)**2
    df2.loc[:, 'weight_4_'+str(models_name[i])] = weights

# WLS Models

In [9]:
# Model 1
y1 = df['retentionrate']

m1 = sm.WLS(y1, sm.add_constant(model1), weights = df['weight_4_model1']).fit() 
df.loc[:, 'predict1'] = m1.predict()

In [10]:
# models 3, 5, 7, 9
y = df2['retentionrate']

m2 = sm.WLS(y, sm.add_constant(model2), weights = df2['weight_4_model2']).fit() 
df2.loc[:,'predict2'] = m2.predict()

m3 = sm.WLS(y, sm.add_constant(model3), weights = df2['weight_4_model3']).fit() 
df2.loc[:,'predict3'] = m3.predict()

m4 = sm.WLS(y, sm.add_constant(model4), weights = df2['weight_4_model4']).fit() 
df2.loc[:,'predict4'] = m4.predict()

m5 = sm.WLS(y, sm.add_constant(model5), weights = df2['weight_4_model5']).fit() 
df2.loc[:,'predict5'] = m5.predict()


In [11]:
# prediction for model 1
x = df[df.name == 'University of New Mexico-Main Campus'].predict1
p1 = math.exp(x)/(1+math.exp(x))
print('Predicted UNM m1''  ' '%0.4f' % p1)

Predicted UNM m1  0.8889


In [12]:
# predicted values WLS
import math
columns = ['predict2', 'predict3', 'predict4', 'predict5']
names = ['m2', 'm3', 'm4', 'm5']

table_unm = df2.loc[df2['name'] == 'University of New Mexico-Main Campus']

for i in range(len(columns)): 
    x = table_unm[columns[i]]
    p = math.exp(x)/(1+math.exp(x))
    print('Predicted UNM '+str(names[i]) + '  ' '%0.4f' % p)

Predicted UNM m2  0.8272
Predicted UNM m3  0.8187
Predicted UNM m4  0.8218
Predicted UNM m5  0.8307


# OLS Models


In [13]:
# models 4, 6, 8, 10 - VCE OLS model
w = df2['pwht']
s = df2['state']
# regression results
m6 = sm.WLS(y1, sm.add_constant(model1), weights = df['pwht']).fit(cov_type='cluster', cov_kwds = {'groups': df['state']}) 
df.loc[:,'predict6'] = m6.predict()

In [14]:
# models 4, 6, 8, 10 - VCE OLS model
m7 = sm.WLS(y, sm.add_constant(model2), weights = w).fit(cov_type='cluster', cov_kwds = {'groups': s}) 
df2.loc[:,'predict7'] = m7.predict()

m8 = sm.WLS(y, sm.add_constant(model3), weights = w).fit(cov_type='cluster', cov_kwds = {'groups': s}) 
df2.loc[:,'predict8'] = m8.predict()

m9 = sm.WLS(y, sm.add_constant(model4), weights = w).fit(cov_type='cluster', cov_kwds = {'groups': s}) 
df2.loc[:,'predict9'] = m9.predict()

m10 = sm.WLS(y, sm.add_constant(model5), weights = w).fit(cov_type='cluster', cov_kwds = {'groups': s}) 
df2.loc[:,'predict10'] = m10.predict()



In [15]:
# prediction for model 6
x = df[df.name == 'University of New Mexico-Main Campus'].predict6
p6 = math.exp(x)/(1+math.exp(x))
print('Predicted UNM m6''  ' '%0.4f' % p6)

Predicted UNM m6  0.8924


In [16]:
#predicted values - OLS
columns = ['predict7', 'predict8', 'predict9', 'predict10']
names = ['m7', 'm8', 'm9', 'm10']

table_unm = df2.loc[df2['name'] == 'University of New Mexico-Main Campus']

for i in range(len(columns)): 
    x = table_unm[columns[i]]
    p = math.exp(x)/(1+math.exp(x))
    print('Predicted UNM '+str(names[i]) + '  ' '%0.4f' % p)

Predicted UNM m7  0.8309
Predicted UNM m8  0.8218
Predicted UNM m9  0.8281
Predicted UNM m10  0.8421


# Table 10

In [17]:
# view table
table = summary_col([m1, m6, m2, m7, m3, m8, m4, m9, m5, m10], stars=True, float_format='%0.3f',
                      regressor_order = ['instruction_FTE', 'academic_support_FTE', 'student_service_FTE',
                      'R1', 'research_FTE', 'SATtoACT_math_25', 'pell_grants', 'stem', 'salary_instruction', 'female', 
                      'white_percent', 'hispanic_percent', 
                      'asian_percent', 'const'],
                      model_names = ['(1)\n WLS', '(2)\n OLS-VCE','(3)\n WLS', '(4)\n OLS-VCE', '(5) \n WLS',
                                    '(6)\n OLS', '(7)\n WLS', '(8)\n OLS-VCE', '(9)\n WLS', '(10) \n OLS-VCE'],
                      info_dict={'R2':lambda x: "{:.2f}".format(x.rsquared),
                                'N':lambda x: "{0:d}".format(int(x.nobs))})

In [18]:
print(table)


                       (1)      (2)       (3)      (4)       (5)       (6)       (7)      (8)       (9)     (10)   
                        WLS    OLS-VCE     WLS    OLS-VCE     WLS       OLS       WLS    OLS-VCE     WLS    OLS-VCE
-------------------------------------------------------------------------------------------------------------------
instruction_FTE      0.013*** 0.008**  0.001     0.003                                           0.001     0.003   
                     (0.003)  (0.003)  (0.003)   (0.004)                                         (0.003)   (0.004) 
academic_support_FTE 0.019*** 0.012*** 0.009**   0.011*** 0.007     0.009***  0.008*    0.009*** 0.010**   0.011***
                     (0.007)  (0.003)  (0.005)   (0.003)  (0.004)   (0.003)   (0.004)   (0.002)  (0.004)   (0.003) 
student_service_FTE  0.085*** 0.106*** 0.032**   0.039**  0.015     0.025     0.013     0.018    0.032**   0.036** 
                     (0.022)  (0.017)  (0.016)   (0.016)  (0.014)   (0.

# Export table

In [19]:
# directory to save tables
import os
os.chdir('C:\\Users\\rbishwakarma\\Desktop\\value_proposition_in_python\\value_proposition\\tables')

results_text = table.as_text()

text_file = open("table8.txt", "w")
text_file.write(results_text)
text_file.close()